<a href="https://colab.research.google.com/github/aidyai/rss-ai/blob/main/rss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%pip install --upgrade --quiet  apify-client langchain-openai langchain
!pip install feedparser newspaper3k openai
!pip install openai==0.28
!pip install replicate

In [2]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document

import feedparser
import requests
from bs4 import BeautifulSoup
import openai
from newspaper import Article

In [3]:
import os
os.environ["APIFY_API_TOKEN"] = ""
os.environ["OPENAI_API_KEY"] = ""


In [8]:
# RSS feed URL
rss_feed_url = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Get the latest just one articles
latest_articles = feed.entries[4:5]

In [9]:
apify = ApifyWrapper()

In [10]:
urls_to_crawl = []
# Loop through the latest articles
for article in latest_articles:
    # Get the title and article URL
    title = article.title
    article_url = article.link
    # Print the article URL
    #print(article_url)
    # Append the article URL to the list of URLs to crawl
    urls_to_crawl.append({"url": article_url})

# Use the list of URLs to call the Apify actor
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": urls_to_crawl},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [11]:
t = loader.load()
t

[Document(page_content='NNPC Exploration and Production Limited (NNPC E&P Ltd.), NNPC Ltd.’s flagship upstream subsidiary, and Natural Oilfield Services Ltd. (NOSL), a subsidiary of Sterling Oil Exploration & Energy Production Company Ltd. (SEEPCO), have announced the successful commencement of oil production at Oil Mining Lease (OML) 13 in Akwa Ibom State, Nigeria.\nThis was revealed by the Chief Corporate Communications Officer, NNPC Ltd, Olufemi Soneye.\nIt reads,” The production, which commenced on the 6th of May 2024, with 6,000 barrels of oil, is expected to be ramped up to 40,000 barrels per day by May 27th, 2024.\n“The first oil flow from OML 13 is a historic milestone in the partnership between NNPC E&P Ltd and NOSL, highlighting their dedication to driving growth and development in Nigeria’s oil and gas sector, which remains a vital component of the nation’s economy.\n“The achievement does not only signify the culmination of rigorous planning and execution by the teams involv

In [ ]:
t = loader.load()
t

[Document(page_content='By Chioma Onuegbu Uyo\nGOVERNOR Umo Eno has continued to demonstrate his readiness, determination, and commitment to fulfilling his contract with the people of Akwa Ibom State including creating job opportunities and enlarging the path for economic prosperity.\nAnd the governor at different fora emphasized that for his administration to achieve this contract there was need to revive the entrepreneurial spirit of the people through comprehensive training and support.\nAccordingly, as an ardent advocate of entrepreneurship development, as soon as he came on board, he hit the ground running, by putting in place variety of initiatives, popular among them is a Business School under the auspices of Ibom-Leadership Entrepreneurial Development (Ibom-LED).\nThe initiative which has its Headquarters at the E-Library Complex along IBB Avenue Uyo has Pastor ImaAbasi Jacob as the team lead and Mrs Ekaette Umoh as the Director General.\nAnd according to Ibom-LED team , the in

In [12]:
import json
# Extracting content from the list
extracted_content = [document.page_content for document in t]

['NNPC Exploration and Production Limited (NNPC E&P Ltd.), NNPC Ltd.’s flagship upstream subsidiary, and Natural Oilfield Services Ltd. (NOSL), a subsidiary of Sterling Oil Exploration & Energy Production Company Ltd. (SEEPCO), have announced the successful commencement of oil production at Oil Mining Lease (OML) 13 in Akwa Ibom State, Nigeria.\nThis was revealed by the Chief Corporate Communications Officer, NNPC Ltd, Olufemi Soneye.\nIt reads,” The production, which commenced on the 6th of May 2024, with 6,000 barrels of oil, is expected to be ramped up to 40,000 barrels per day by May 27th, 2024.\n“The first oil flow from OML 13 is a historic milestone in the partnership between NNPC E&P Ltd and NOSL, highlighting their dedication to driving growth and development in Nigeria’s oil and gas sector, which remains a vital component of the nation’s economy.\n“The achievement does not only signify the culmination of rigorous planning and execution by the teams involved but also represents

In [13]:
import openai
# Send the content to OpenAI to get topic and category tags
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=extracted_content,
    max_tokens=50,  # Adjust as needed
    n=1,  # Number of completions to generate
    stop=["\n"],  # Stop completion at the end of the text
    temperature=0.5,  # Adjust as needed
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    logit_bias={},
    logprobs=0,
    echo=True  # Print the input prompt along with the completion
)


# USING REPLICATE API TO CALL POWERFUL LARGE LANGUAGE MODELS

In [17]:
import replicate

In [26]:
import os
import replicate
from replicate.client import Client

replicate = Client(api_token="")

#Set the REPLICATE_API_TOKEN environment variable
#os.environ["REPLICATE_API_TOKEN"] = "r8_A8YqxkWaPljHoVOSzEH7zhpjHyACd9i1EJFoM"
#api = replicate.Client(api_token=os.environ["REPLICATE_API_TOKEN"])

## USING MISTRAL 8x7B via Replicate Api

In [31]:
for event in replicate.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSIATAT THAT CAN GENERATE topic tags AND  article category tags",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "prompt_template": "<s>[INST] {prompt} [/INST] ",
        "presence_penalty": 0
    },
):
    print(str(event), end="")

1. Topic Tags: Oil Production, NNPC, Natural Oilfield Services Ltd, OML 1

## USING METAS LLAMA 3 Via Replicate Api

In [33]:
for event in replicate.stream(
    "meta/meta-llama-3-70b-instruct",
    input={
        "top_k": 0,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "max_tokens": 650,
        "min_tokens": 0,
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSITANT THAT CAN GENERATE topic tags AND article category tags",
        "length_penalty": 1,
        "stop_sequences": "<|end_of_text|>,<|eot_id|>",
        "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "presence_penalty": 1.15,
        "log_performance_metrics": False
    },
):
    print(str(event), end="")

Here are five topic tags and five category tags for the given article:

**Topic Tags:**

1. Oil Production
2. Nigeria Energy Sector
3. NNPC E&P Ltd
4. Natural Oilfield Services Ltd
5. Oil Mining Lease (OML) 13

**Category Tags:**

1. Business
2. Energy
3. Nigeria News
4. Oil and Gas
5. Economy